In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Small_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Medium_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_accounts.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_Trans.csv
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Large_Patterns.txt
/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/LI-Medium_Patterns.txt
/kaggle/in

In [2]:
# !pip install dgl

In [3]:
!pip install --no-cache-dir dgl==1.1.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 65.0 MB/s eta 0:00:00


In [4]:
import dgl
import torch
from collections import defaultdict
import pandas as pd

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [5]:
df_account = pd.read_csv("/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_accounts.csv")
df_account.head(20)

,Bank Name,Bank ID,Account Number,Entity ID,Entity Name
0,Portugal Bank #4507,331579,80B779D80,80062E240,Sole Proprietorship #50438
1,Canada Bank #27,210,809D86900,800C998A0,Corporation #33520
2,UK Bank #33,21884,80812BE00,800C47F50,Partnership #35397
3,Germany Bank #4815,32742,81047F300,80096F0B0,Corporation #48813
4,National Bank of Harrisburg,127390,80BD8CF00,800FB8760,Corporation #889
5,Spain Bank #439,224555,80F269580,80064EB20,Sole Proprietorship #42987
6,Savings Bank of Omaha,32013,80E6E8680,800CCEDE0,Sole Proprietorship #20855
7,Brazil Bank #39,335355,80CDEFD80,800D88F10,Partnership #36822
8,Mexico Bank #16,1132,80B723600,800D3F760,Partnership #36511
9,Russia Bank #39,217824,806B17000,800B44480,Partnership #33830


In [6]:
df_trans = pd.read_csv("/kaggle/input/ibm-transactions-for-anti-money-laundering-aml/HI-Small_Trans.csv")
df_trans.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [7]:
df_joined = pd.merge(
    df_trans,
    df_account,
    how='left',
    left_on='Account',
    right_on='Account Number'
)
df_joined.drop(columns=['Account Number'], inplace=True)

In [8]:
df_joined.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,Bank Name,Bank ID,Entity ID,Entity Name
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0,National Bank of Laramie,10,800D232D0,Partnership #1
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0,Sappo Cooperative Bank,3208,8008EEA70,Partnership #2
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0,National Bank of Fort Wayne,3209,800FBB3A0,Partnership #3
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0,National Bank of the East,12,800C0EF20,Sole Proprietorship #1
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0,National Bank of Laramie,10,800C3EC10,Partnership #4


In [9]:
from_bank_counts_df = df_joined['From Bank'].value_counts().reset_index()
from_bank_counts_df.columns = ['From Bank', 'Jumlah']
print(from_bank_counts_df)

       From Bank  Jumlah
0             70  449859
1             10   81629
2             12   79754
3              1   62211
4             15   52511
...          ...     ...
30465     351005       1
30466     351022       1
30467     349350       1
30468     340466       1
30469     336827       1

[30470 rows x 2 columns]


In [10]:
df_joined = df_joined.drop(columns=['Bank Name', 'Receiving Currency', 'Payment Currency'])
top_5_banks = [70, 10, 12, 1, 15]

df_70 = df_joined[df_joined['From Bank'] == 70]
df_10 = df_joined[df_joined['From Bank'] == 10]
df_12 = df_joined[df_joined['From Bank'] == 12]
df_1  = df_joined[df_joined['From Bank'] == 1]
df_15 = df_joined[df_joined['From Bank'] == 15]


In [11]:
df_15.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Amount Paid', 'Payment Format', 'Is Laundering',
       'Bank ID', 'Entity ID', 'Entity Name'],
      dtype='object')

In [12]:
df_70.to_csv('/kaggle/working/df_Oasis_Thrift.csv', index=True)
df_10.to_csv('/kaggle/working/df_Laramie.csv', index=True)
df_12.to_csv('/kaggle/working/df_East.csv', index=True)
df_1.to_csv('/kaggle/working/df_Arbor_Savings.csv', index=True)
df_15.to_csv('/kaggle/working/df_Japan.csv', index=True)

In [13]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
import pandas as pd
import torch
import dgl

def preprocess_bank_graph(df_bank):
    edge_dict = defaultdict(list)
    edge_feats = defaultdict(dict)

    # Konversi Timestamp ke datetime dan ke detik sejak awal
    df_bank = df_bank.copy()  # untuk menghindari SettingWithCopyWarning
    df_bank['Timestamp'] = pd.to_datetime(df_bank['Timestamp'])
    df_bank['Timestamp_Seconds'] = (df_bank['Timestamp'] - df_bank['Timestamp'].min()).dt.total_seconds()

    # Unique ID maps
    account_ids = pd.Index(df_bank['Account'].tolist() + df_bank['Account.1'].tolist()).unique()
    account_id_map = {k: i for i, k in enumerate(account_ids)}
    bank_ids = pd.Index(df_bank['From Bank'].tolist()).unique()
    bank_id_map = {k: i for i, k in enumerate(bank_ids)}
    entity_ids = pd.Index(df_bank['Entity ID'].tolist()).unique()
    entity_id_map = {k: i for i, k in enumerate(entity_ids)}

    # Encode payment format
    le_payment = LabelEncoder()
    df_bank['Payment Format Enc'] = le_payment.fit_transform(df_bank['Payment Format'])

    transfer_edge_feat = {
        'amount': [],
        'timestamp': [],
        'payment_format': [],
        'label': []
    }

    for idx, row in df_bank.iterrows():
        src = account_id_map[row['Account']]
        dst = account_id_map[row['Account.1']]
        edge_dict[('account', 'transfers', 'account')].append((src, dst))

        # Tambahkan fitur edge
        transfer_edge_feat['amount'].append(row['Amount Paid'])
        transfer_edge_feat['timestamp'].append(row['Timestamp_Seconds'])  # now numeric
        transfer_edge_feat['payment_format'].append(row['Payment Format Enc'])
        transfer_edge_feat['label'].append(row['Is Laundering'])

        # Edge ke bank
        edge_dict[('account', 'belongs_to', 'bank')].append((src, bank_id_map[row['From Bank']]))
        # Edge ke entitas
        edge_dict[('account', 'represents', 'entity')].append((src, entity_id_map[row['Entity ID']]))

    # Build graph
    graph_data = {}
    for etype, edges in edge_dict.items():
        srcs, dsts = zip(*edges)
        graph_data[etype] = (torch.tensor(srcs), torch.tensor(dsts))

    g = dgl.heterograph(graph_data)

    # Tambahkan edge features untuk 'transfers' edge
    g.edges['transfers'].data['amount'] = torch.tensor(transfer_edge_feat['amount'], dtype=torch.float32)
    g.edges['transfers'].data['timestamp'] = torch.tensor(transfer_edge_feat['timestamp'], dtype=torch.float32)
    g.edges['transfers'].data['payment_format'] = torch.tensor(transfer_edge_feat['payment_format'], dtype=torch.int64)
    g.edges['transfers'].data['label'] = torch.tensor(transfer_edge_feat['label'], dtype=torch.float32)

    # Simpan encoder dan id_map kalau mau digunakan lagi
    g.account_id_map = account_id_map
    g.bank_id_map = bank_id_map
    g.entity_id_map = entity_id_map
    g.label_encoder = le_payment

    return g


In [14]:
bank_graphs = {
    1: preprocess_bank_graph(df_1),
    10: preprocess_bank_graph(df_10),
    12: preprocess_bank_graph(df_12),
    15: preprocess_bank_graph(df_15),
    70: preprocess_bank_graph(df_70),
}


In [15]:
for bank_id, graph in bank_graphs.items():
    save_path = f"/kaggle/working/graph_bank_{bank_id}.bin"
    dgl.save_graphs(save_path, [graph])
    print(f"Saved graph for bank {bank_id} to {save_path}")

Saved graph for bank 1 to /kaggle/working/graph_bank_1.bin
Saved graph for bank 10 to /kaggle/working/graph_bank_10.bin
Saved graph for bank 12 to /kaggle/working/graph_bank_12.bin
Saved graph for bank 15 to /kaggle/working/graph_bank_15.bin
Saved graph for bank 70 to /kaggle/working/graph_bank_70.bin


In [16]:
# loaded_graphs = {}
# for bank_id in [1, 10, 12, 15, 70]:
#     g, _ = dgl.load_graphs(f"/kaggle/working/graph_bank_{bank_id}.bin")
#     loaded_graphs[bank_id] = g[0]
